In [36]:
# !pip install psycopg2-binary
# !pip install SQLAlchemy

     ---------------------------------------- 2.0/2.0 MB 5.4 MB/s eta 0:00:00
     -------------------------------------- 192.5/192.5 kB 5.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [109]:
import pandas as pd
from uuid import uuid4
import psycopg2
from sqlalchemy import create_engine


In [110]:
# connect to local postgresql instance
HOST = 'localhost'
PORT = 5432
USERNAME = 'postgres'
PASSWORD = 'root'
DBNAME = 'gadgetry'
def get_connection(host, port, user,password,dbname):
    connection_str = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
    engine = create_engine(connection_str)
    return engine

conn = get_connection(HOST,PORT,USERNAME,PASSWORD,DBNAME)

In [111]:
# read csv
# download data from :https://drive.google.com/file/d/1aXzzmRIFOdd5v7P1BgCLOqQLsATqFWBc/view?usp=share_link
csv_raw_data = pd.read_csv('DatafinitiElectronicsProductData.csv')
csv_raw_data.head()

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,imageURLs,...,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,upc,weight
0,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,8.900000e+11,https://i5.walmartimages.com/asr/2a41f6f0-844e...,...,True,0.0,5.0,http://reviews.bestbuy.com/3545/4562009/review...,"This keyboard is very easy to type on, but the...",Love the fingerprint reader,JNH1,https://www.walmart.com/ip/Microsoft-Surface-P...,8.900000e+11,1.1 pounds
1,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,8.900000e+11,https://i5.walmartimages.com/asr/2a41f6f0-844e...,...,True,0.0,4.0,http://reviews.bestbuy.com/3545/4562009/review...,It's thin and light. I can type pretty easily ...,Nice,Appa,https://www.walmart.com/ip/Microsoft-Surface-P...,8.900000e+11,1.1 pounds
2,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,8.900000e+11,https://i5.walmartimages.com/asr/2a41f6f0-844e...,...,True,0.0,4.0,http://reviews.bestbuy.com/3545/4562009/review...,I love the new design the keys are spaced well...,New,Kman,https://www.walmart.com/ip/Microsoft-Surface-P...,8.900000e+11,1.1 pounds
3,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,8.900000e+11,https://i5.walmartimages.com/asr/2a41f6f0-844e...,...,True,0.0,5.0,http://reviews.bestbuy.com/3545/4562009/review...,Attached easily and firmly. Has a nice feel. A...,Nice keyboard,UpstateNY,https://www.walmart.com/ip/Microsoft-Surface-P...,8.900000e+11,1.1 pounds
4,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,8.900000e+11,https://i5.walmartimages.com/asr/2a41f6f0-844e...,...,True,0.0,5.0,http://reviews.bestbuy.com/3545/4562009/review...,"Our original keyboard was okay, but did not ha...",Nice improvement,Glickster,https://www.walmart.com/ip/Microsoft-Surface-P...,8.900000e+11,1.1 pounds


In [112]:
csv_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7299 entries, 0 to 7298
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   7299 non-null   object 
 1   asins                7299 non-null   object 
 2   brand                7299 non-null   object 
 3   categories           7299 non-null   object 
 4   colors               5280 non-null   object 
 5   dateAdded            7299 non-null   object 
 6   dateUpdated          7299 non-null   object 
 7   dimension            6090 non-null   object 
 8   ean                  2951 non-null   float64
 9   imageURLs            7299 non-null   object 
 10  keys                 7299 non-null   object 
 11  manufacturer         4632 non-null   object 
 12  manufacturerNumber   7299 non-null   object 
 13  name                 7299 non-null   object 
 14  primaryCategories    7299 non-null   object 
 15  reviews.date         7238 non-null   o

In [134]:
# PROCESSING PRODUCTS
COLUMNS_TO_SELECT = [
    'name',
    'brand',
    'imageURLs',
    'dateAdded',
    'dimension',
]

PRODUCT_COLUMNS = {
    'name':'productName',
    'brand':'productBrand',
    'dateAdded':'productDateAdded',
    'dimension':'productDimensions',
    'imageURLs':'productImageUrl',
    'user':'Username_id'
}

REVIEW_COLUMN_TO_SELECT = [
    'reviews.text',
    'reviews.date',
    'reviews.rating',
    'name'
]

REVIEW_COLUMNS = {
    'reviews.text':'reviewText',
    'reviews.date':'reviewDate',
    'reviews.rating':'reviewRating',
    'name':'productName_id'
}
USER_TABLE_NAME = 'api_user'
UNIQUE_USER_ID = 'DEV'
UNIQUE_USER_PASSWORD='pass'

REVIEW_TABLE_NAME = 'api_review'
PRODUCT_TABLE_NAME = 'api_product'

In [114]:
# INSERTING DUMMY USER ENTRy
user_entry = pd.DataFrame({
    'userName': [UNIQUE_USER_ID],
    'userPassword':[UNIQUE_USER_PASSWORD],
    'User': [UNIQUE_USER_ID]
})
try:
    result = user_entry.to_sql(name=USER_TABLE_NAME,con=conn,if_exists='append',index=False)
except Exception as e:
    print(e)

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "api_user_pkey"
DETAIL:  Key ("userName")=(DEV) already exists.

[SQL: INSERT INTO api_user ("userName", "userPassword", "User") VALUES (%(userName)s, %(userPassword)s, %(User)s)]
[parameters: {'userName': 'DEV', 'userPassword': 'pass', 'User': 'DEV'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [115]:
products_data = csv_raw_data[COLUMNS_TO_SELECT].drop_duplicates(keep='first')

In [116]:
# parse_date
products_data['dateAdded'] = pd.to_datetime(products_data['dateAdded'],format = '%Y-%m-%dT%H:%M:%SZ').dt.date
products_data['user'] = UNIQUE_USER_ID
products_data['dimension'] = products_data['dimension'].fillna('NA')
# rename columns
products_data.rename(columns=PRODUCT_COLUMNS,inplace=True)



In [119]:
try:
    result = products_data.to_sql(name=PRODUCT_TABLE_NAME,con=conn,if_exists='append',index=False)
except Exception as e:
    print("LMAO")

LMAO


In [145]:
# PROCESSIG REVIEWS
reviews_data = csv_raw_data[REVIEW_COLUMN_TO_SELECT]
reviews_data = reviews_data.dropna(axis=0,how='any')
reviews_data['reviews.date'] = reviews_data['reviews.date'].str.slice(start=0,stop = 10)
reviews_data['reviews.date'] = reviews_data['reviews.date'].fillna('2022-10-22')
reviews_data['reviews.date'] = pd.to_datetime(reviews_data['reviews.date'],format = '%Y-%m-%d').dt.date
reviews_data['username_id'] = UNIQUE_USER_ID
review_ids = [uuid4() for _ in range(len(reviews_data))]
reviews_data['ReviewId'] = review_ids
reviews_data['reviews.rating'] = pd.to_numeric(reviews_data['reviews.rating'])
reviews_data['reviews.rating'] = reviews_data['reviews.rating'].fillna(0)
reviews_data.rename(columns=REVIEW_COLUMNS,inplace=True)

In [146]:
try:
    result = reviews_data.to_sql(name=REVIEW_TABLE_NAME,con=conn,if_exists='append',index=False)
except Exception as e:
    print(e)